In [2]:
# ==========================================
# 📘 Sea Animals Image Classification (Vision Transformer)
# ==========================================
# Vollautomatisches Training + Evaluation + Prediction
# Erstellt für Amr Selck
# ==========================================

# ✅ Falls nötig, Bibliotheken installieren:
# !pip install torch torchvision matplotlib scikit-learn pillow

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from pathlib import Path
import numpy as np
import time
import logging
from datetime import datetime
from PIL import Image
import random

# ==========================================
# 1️⃣ Setup & Logging
# ==========================================
ROOT = Path().resolve()
TRAIN_DIR = ROOT / "FishDataset/train"
VAL_DIR = ROOT / "FishDataset/val"
MODELS_DIR = ROOT / "models"
LOGS_DIR = ROOT / "logs"
MODELS_DIR.mkdir(exist_ok=True)
LOGS_DIR.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = LOGS_DIR / f"vit_train_{timestamp}.log"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler(log_file, encoding="utf-8"), logging.StreamHandler()],
)
logger = logging.getLogger("vit_notebook")
logger.info("📘 Training gestartet")

# ==========================================
# 2️⃣ Device Detection
# ==========================================
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
logger.info(f"🔥 Device aktiv: {device}")

# ==========================================
# 3️⃣ Dataset & Transform
# ==========================================
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=transform_train)
val_dataset = datasets.ImageFolder(VAL_DIR, transform=transform_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

classes = train_dataset.classes
num_classes = len(classes)
logger.info(f"📊 {num_classes} Klassen erkannt: {classes}")

# ==========================================
# 4️⃣ Modell (Vision Transformer)
# ==========================================
model = models.vit_b_16(weights="IMAGENET1K_V1")
for p in model.parameters():
    p.requires_grad = False

model.heads = nn.Sequential(
    nn.Linear(model.heads.head.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes)
)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.heads.parameters(), lr=3e-4)

# ==========================================
# 5️⃣ Training
# ==========================================
EPOCHS = 8
train_acc, val_acc, val_loss_list = [], [], []
best_acc = 0.0
start_time = time.time()

for epoch in range(EPOCHS):
    model.train()
    correct, total = 0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_accuracy = 100 * correct / total
    train_acc.append(train_accuracy)

    # Validation
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    val_accuracy = 100 * correct / total
    val_loss /= len(val_loader)
    val_acc.append(val_accuracy)
    val_loss_list.append(val_loss)

    logger.info(f"Epoch {epoch+1}: Train={train_accuracy:.2f}% | Val={val_accuracy:.2f}% | Loss={val_loss:.4f}")

    if val_accuracy > best_acc:
        best_acc = val_accuracy
        torch.save(model.state_dict(), MODELS_DIR / "vit_best.pth")

# ==========================================
# 6️⃣ Evaluation
# ==========================================
duration = (time.time() - start_time) / 60
logger.info(f"✅ Training abgeschlossen in {duration:.1f} Min. Beste Val-Acc: {best_acc:.2f}%")

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(xticks_rotation="vertical", cmap="viridis")
plt.tight_layout()
plt.savefig(MODELS_DIR / "vit_confusion_matrix.png")
plt.close()

# Accuracy Plot
plt.figure(figsize=(8,5))
plt.plot(train_acc, label="Train Acc")
plt.plot(val_acc, label="Val Acc")
plt.legend()
plt.title("ViT Accuracy Verlauf")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.grid(True)
plt.savefig(MODELS_DIR / "vit_accuracy.png")
plt.close()

# Loss Plot
plt.figure(figsize=(8,5))
plt.plot(val_loss_list, label="Validation Loss", color="orange")
plt.legend()
plt.title("ViT Loss Verlauf")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.savefig(MODELS_DIR / "vit_loss.png")
plt.close()

logger.info("📈 Visualisierungen gespeichert in /models/")
print("Training abgeschlossen ✅")

# ==========================================
# 7️⃣ Prediction (Einzelbild-Test)
# ==========================================
# Lade das gespeicherte Modell
model.load_state_dict(torch.load(MODELS_DIR / "vit_best.pth", map_location=device))
model.eval()

# Wähle zufälliges Bild aus Validation-Ordner
sample = random.choice(list(VAL_DIR.glob("*/*.jpg")))
img = Image.open(sample).convert("RGB")

# Vorbereitung für Prediction
transform_pred = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
x = transform_pred(img).unsqueeze(0).to(device)

# Vorhersage
with torch.no_grad():
    preds = model(x)
    probs = torch.nn.functional.softmax(preds, dim=1)
    pred = probs.argmax(dim=1).item()
    conf = probs[0][pred].item() * 100

# Ergebnis anzeigen
plt.imshow(np.array(img))
plt.title(f"✅ Ergebnis: {classes[pred]} ({conf:.2f}% Konfidenz)")
plt.axis("off")
plt.show()

logger.info(f"🔍 Bild: {sample}")
logger.info(f"✅ Ergebnis: {classes[pred]} ({conf:.2f}% Konfidenz)")


2025-11-06 22:58:55,416 - INFO - 📘 Training gestartet
2025-11-06 22:58:55,417 - INFO - 🔥 Device aktiv: cpu
2025-11-06 22:58:55,476 - INFO - 📊 22 Klassen erkannt: ['Clams', 'Corals', 'Crabs', 'Dolphin', 'Eel', 'Fish', 'Jelly Fish', 'Lobster', 'Nudibranchs', 'Octopus', 'Otter', 'Penguin', 'Puffers', 'Sea Rays', 'Sea Urchins', 'Seahorse', 'Seal', 'Sharks', 'Shrimp', 'Squid', 'Starfish', 'Turtle_Tortoise']


KeyboardInterrupt: 

In [ ]:
# ==========================================
# 📘 Sea Animals Image Classification (Vision Transformer)
# ==========================================
# Vollautomatisches Training + Evaluation + Prediction
# Erstellt für Amr Selck
# ==========================================

# ✅ Falls nötig, Bibliotheken installieren:
# !pip install torch torchvision matplotlib scikit-learn pillow

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from pathlib import Path
import numpy as np
import time
import logging
from datetime import datetime
from PIL import Image
import random

# ==========================================
# 1️⃣ Setup & Logging
# ==========================================
ROOT = Path().resolve()
TRAIN_DIR = ROOT / "FishDataset/train"
VAL_DIR = ROOT / "FishDataset/val"
MODELS_DIR = ROOT / "models"
LOGS_DIR = ROOT / "logs"
MODELS_DIR.mkdir(exist_ok=True)
LOGS_DIR.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = LOGS_DIR / f"vit_train_{timestamp}.log"

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler(log_file, encoding="utf-8"), logging.StreamHandler()],
)
logger = logging.getLogger("vit_notebook")
logger.info("📘 Training gestartet")

# ==========================================
# 2️⃣ Device Detection
# ==========================================
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
logger.info(f"🔥 Device aktiv: {device}")

# ==========================================
# 3️⃣ Dataset & Transform
# ==========================================
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=transform_train)
val_dataset = datasets.ImageFolder(VAL_DIR, transform=transform_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

classes = train_dataset.classes
num_classes = len(classes)
logger.info(f"📊 {num_classes} Klassen erkannt: {classes}")

# ==========================================
# 4️⃣ Modell (Vision Transformer)
# ==========================================
model = models.vit_b_16(weights="IMAGENET1K_V1")
for p in model.parameters():
    p.requires_grad = False

model.heads = nn.Sequential(
    nn.Linear(model.heads.head.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, num_classes)
)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.heads.parameters(), lr=3e-4)

# ==========================================
# 5️⃣ Training
# ==========================================
EPOCHS = 8
train_acc, val_acc, val_loss_list = [], [], []
best_acc = 0.0
start_time = time.time()

for epoch in range(EPOCHS):
    model.train()
    correct, total = 0, 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_accuracy = 100 * correct / total
    train_acc.append(train_accuracy)

    # Validation
    model.eval()
    correct, total, val_loss = 0, 0, 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    val_accuracy = 100 * correct / total
    val_loss /= len(val_loader)
    val_acc.append(val_accuracy)
    val_loss_list.append(val_loss)

    logger.info(f"Epoch {epoch+1}: Train={train_accuracy:.2f}% | Val={val_accuracy:.2f}% | Loss={val_loss:.4f}")

    if val_accuracy > best_acc:
        best_acc = val_accuracy
        torch.save(model.state_dict(), MODELS_DIR / "vit_best.pth")

# ==========================================
# 6️⃣ Evaluation
# ==========================================
duration = (time.time() - start_time) / 60
logger.info(f"✅ Training abgeschlossen in {duration:.1f} Min. Beste Val-Acc: {best_acc:.2f}%")

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(xticks_rotation="vertical", cmap="viridis")
plt.tight_layout()
plt.savefig(MODELS_DIR / "vit_confusion_matrix.png")
plt.close()

# Accuracy Plot
plt.figure(figsize=(8,5))
plt.plot(train_acc, label="Train Acc")
plt.plot(val_acc, label="Val Acc")
plt.legend()
plt.title("ViT Accuracy Verlauf")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.grid(True)
plt.savefig(MODELS_DIR / "vit_accuracy.png")
plt.close()

# Loss Plot
plt.figure(figsize=(8,5))
plt.plot(val_loss_list, label="Validation Loss", color="orange")
plt.legend()
plt.title("ViT Loss Verlauf")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.savefig(MODELS_DIR / "vit_loss.png")
plt.close()

logger.info("📈 Visualisierungen gespeichert in /models/")
print("Training abgeschlossen ✅")

# ==========================================
# 7️⃣ Prediction (Einzelbild-Test)
# ==========================================
# Lade das gespeicherte Modell
model.load_state_dict(torch.load(MODELS_DIR / "vit_best.pth", map_location=device))
model.eval()

# Wähle zufälliges Bild aus Validation-Ordner
sample = random.choice(list(VAL_DIR.glob("*/*.jpg")))
img = Image.open(sample).convert("RGB")

# Vorbereitung für Prediction
transform_pred = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
x = transform_pred(img).unsqueeze(0).to(device)

# Vorhersage
with torch.no_grad():
    preds = model(x)
    probs = torch.nn.functional.softmax(preds, dim=1)
    pred = probs.argmax(dim=1).item()
    conf = probs[0][pred].item() * 100

# Ergebnis anzeigen
plt.imshow(np.array(img))
plt.title(f"✅ Ergebnis: {classes[pred]} ({conf:.2f}% Konfidenz)")
plt.axis("off")
plt.show()

logger.info(f"🔍 Bild: {sample}")
logger.info(f"✅ Ergebnis: {classes[pred]} ({conf:.2f}% Konfidenz)")
